In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from IPython.display import display
from IPython.display import display_html
from sklearn.feature_selection import RFECV
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [49]:
train = pd.read_csv('train.csv')
test  = pd.read_csv('test.csv')
df_data  =train.append(test)


In [47]:
train[["Embarked","Survived"]].groupby("Embarked",as_index=False).mean().sort_values(['Survived'], ascending=False)

,Embarked,Survived
0,C,0.553571
1,Q,0.389610
2,S,0.336957


###  將票價分組

In [42]:
df_data['Fare'] = df_data['Fare'].fillna(df_data['Fare'].median())
df_data['FareBin_5']   = pd.qcut(df_data['Fare'],5)

### 給予區間標籤 and 性別更改回int64 EX male->1 

In [43]:
label = LabelEncoder()
df_data['FareBin_Code_5'] = label.fit_transform(df_data['FareBin_5'])
df_data['Sex']            = label.fit_transform(df_data['Sex'])

In [44]:
X_train = df_data[:len(train)].iloc[:,[2,4,13]]
X_test  = df_data[len(train):].iloc[:,[2,4,13]]
Y_train = df_data[:len(train)].iloc[:,1].astype('int64')

In [45]:
from sklearn.ensemble import RandomForestClassifier
#RFC = RandomForestClassifier(random_state=2,n_estimators=250,min_samples_split=20,oob_score=True)
RFC   = RFECV(RandomForestClassifier(n_estimators=250,min_samples_split=20),cv=10,n_jobs=-1)
RFC.fit(X_train, Y_train)
Y_pred = RFC.predict(X_test)
randomfroest_score = round(RFC.score(X_train, Y_train) * 100, 2)
randomfroest_score

81.03

In [46]:
index = pd.RangeIndex(start =892, stop =1310 , step =1)
submission = pd.DataFrame(
{"PassengerId" : index ,
 "Survived"    : Y_pred}
)
submission.to_csv("titanic90.csv", index= False)